# Predicción de crímenes
### Trabajo Fin de Máster para el Máster Universitario en Ciencia de Datos
### Universitat Oberta de Catalunya
### Realizado por Álvaro Pavón Díaz

## Parte 2: Modelado y evaluación de los datos
Durante este Jupyter Notebook nos dedicaremos a realizar el entrenamiento de los conjunto de datos sobre diferentes métodos.

Para ello lo primero se va a proceder a importar las diferentes clases que se necesitarán a lo largo de este notebook y la importación de los archivos que se va a utilizar. Además de mostrar que los proceso realizados a través de TensorFlow van a ser acelerados mediante GPU (GeForce RTX 2070 SUPER)

In [1]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time_function
import tensorflow as tf
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1" 

print(device_lib.list_local_devices())
print(tf.version)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1794315526548503137
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6586313605
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12686134853631233297
physical_device_desc: "device: 0, name: GeForce RTX 2070 SUPER, pci bus id: 0000:2d:00.0, compute capability: 7.5"
]
<module 'tensorflow_core._api.v2.version' from 'C:\\Users\\darkc\\AppData\\Roaming\\Python\\Python37\\site-packages\\tensorflow_core\\_api\\v2\\version\\__init__.py'>


In [2]:
crimes = pd.read_csv ('crimes_clean.csv', sep='""', delimiter=',', engine='python')
crimes.head(5)

,OFFENSE_CODE_GROUP,DISTRICT,SHOOTING,YEAR,MONTH,DAY_OF_WEEK,HOUR,STREET,Lat,Long,IS_NIGHT,DAY_OF_MONTH
0,Disorderly Conduct,E18,0.0,2018,10,Wednesday,20,ARLINGTON ST,42.262608,-71.121186,1,3
1,Property Lost,D14,0.0,2018,8,Thursday,20,ALLSTON ST,42.352111,-71.135311,1,30
2,Other,B2,0.0,2018,10,Wednesday,19,DEVON ST,42.308126,-71.076930,1,3
3,Aggravated Assault,A1,0.0,2018,10,Wednesday,20,CAMBRIDGE ST,42.359454,-71.059648,1,3
4,Aircraft,A7,0.0,2018,10,Wednesday,20,PRESCOTT ST,42.375258,-71.024663,1,3


In [3]:
crimes_time = pd.read_csv ('crimes_clean_time_2.csv', sep='""', delimiter=',', engine='python')
crimes_time.head(5)

,OFFENSE_CODE_GROUP,DISTRICT,SHOOTING,YEAR,MONTH,DAY_OF_WEEK,HOUR,STREET,Lat,Long,IS_NIGHT,DAY_OF_MONTH,TIME_X,TIME_Y,Lat_M,Long_M
0,Desorden,E18,0.0,2018,10,Wednesday,20,ARLINGTON ST,42.262608,-71.121186,1,3,0.518198,0.000331,0.185665,0.267386
1,Otros,D14,0.0,2018,8,Thursday,20,ALLSTON ST,42.352111,-71.135311,1,30,0.239106,0.073462,0.736023,0.201687
2,Otros,B2,0.0,2018,10,Wednesday,19,DEVON ST,42.308126,-71.076930,1,3,0.517840,0.000318,0.465558,0.473233
3,Desorden,A1,0.0,2018,10,Wednesday,20,CAMBRIDGE ST,42.359454,-71.059648,1,3,0.518198,0.000331,0.781170,0.553614
4,Otros,A7,0.0,2018,10,Wednesday,20,PRESCOTT ST,42.375258,-71.024663,1,3,0.518198,0.000331,0.878350,0.716335


In [4]:
crimes_time=crimes_time[['OFFENSE_CODE_GROUP', 'SHOOTING','YEAR','MONTH','DAY_OF_MONTH','DAY_OF_WEEK','HOUR','IS_NIGHT','Lat', 'Lat_M', 'Long','Long_M', 'TIME_X', 'TIME_Y', 'DISTRICT']]
crimes_time.head(5)

,OFFENSE_CODE_GROUP,SHOOTING,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,HOUR,IS_NIGHT,Lat,Lat_M,Long,Long_M,TIME_X,TIME_Y,DISTRICT
0,Desorden,0.0,2018,10,3,Wednesday,20,1,42.262608,0.185665,-71.121186,0.267386,0.518198,0.000331,E18
1,Otros,0.0,2018,8,30,Thursday,20,1,42.352111,0.736023,-71.135311,0.201687,0.239106,0.073462,D14
2,Otros,0.0,2018,10,3,Wednesday,19,1,42.308126,0.465558,-71.076930,0.473233,0.517840,0.000318,B2
3,Desorden,0.0,2018,10,3,Wednesday,20,1,42.359454,0.781170,-71.059648,0.553614,0.518198,0.000331,A1
4,Otros,0.0,2018,10,3,Wednesday,20,1,42.375258,0.878350,-71.024663,0.716335,0.518198,0.000331,A7


In [5]:
crimes_times = crimes_time.sort_values(['YEAR', 'MONTH', 'DAY_OF_MONTH', 'HOUR'], ascending=[True, True, True, True])
crimes_times.head(5)

,OFFENSE_CODE_GROUP,SHOOTING,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,HOUR,IS_NIGHT,Lat,Lat_M,Long,Long_M,TIME_X,TIME_Y,DISTRICT
127685,Sexual,0.0,2015,6,15,Monday,0,1,42.291093,0.360820,-71.065945,0.524324,0.022469,0.648202,C11
242709,Otros,0.0,2015,6,15,Monday,0,1,42.283634,0.314958,-71.082813,0.445868,0.022469,0.648202,B3
290054,Fraude,0.0,2015,6,15,Monday,0,1,42.360205,0.785788,-71.056208,0.569614,0.022469,0.648202,A1
297104,Otros,0.0,2015,6,15,Monday,0,1,42.293606,0.376273,-71.071887,0.496690,0.022469,0.648202,C11
301293,Hurto,0.0,2015,6,15,Monday,0,1,42.300217,0.416923,-71.080979,0.454397,0.022469,0.648202,B3


In [6]:
crimes_time_one = crimes_time
le_dw = LabelEncoder()
dw_labels = le_dw.fit_transform(crimes_time_one['DAY_OF_WEEK'])
crimes_time_one['DAY_OF_WEEK_label'] = dw_labels

one_dw = OneHotEncoder()
days_of_week = one_dw.fit_transform(crimes_time_one[['DAY_OF_WEEK_label']]).toarray()
days_of_week_labels = list(le_dw.classes_)
days_of_week_df = pd.DataFrame(days_of_week, columns=days_of_week_labels)



In [7]:
le_ocg = LabelEncoder()
ocg_labels = le_ocg.fit_transform(crimes_time_one['OFFENSE_CODE_GROUP'])
crimes_time_one['OFFENSE_CODE_label'] = ocg_labels

one_ocg = OneHotEncoder()
ocg = one_ocg.fit_transform(crimes_time_one[['OFFENSE_CODE_label']]).toarray()
ocg_labels = list(le_ocg.classes_)
ocg_df = pd.DataFrame(ocg, columns=ocg_labels)

In [8]:
#le_year = LabelEncoder()
#year_labels = le_year.fit_transform(crimes_time_one['YEAR'])
#crimes_time_one['year_label'] = year_labels

#one_year = OneHotEncoder()
#year = one_year.fit_transform(crimes_time_one[['year_label']]).toarray()
#year_labels = list(le_year.classes_)
#year_df = pd.DataFrame(year, columns=year_labels)

In [9]:
le_d = LabelEncoder()
d_labels = le_d.fit_transform(crimes_time_one['DISTRICT'])
crimes_time_one['DISTRICT_labels'] = d_labels

In [10]:
crimes_time_one = pd.concat([ocg_df,days_of_week_df, crimes_time_one], axis=1)

crimes_time_one.drop('DAY_OF_WEEK_label', 1, inplace= True)
crimes_time_one.drop('OFFENSE_CODE_label', 1, inplace= True)
crimes_time_one.drop('DAY_OF_WEEK', 1, inplace= True)
crimes_time_one.drop('OFFENSE_CODE_GROUP', 1, inplace= True)
#crimes_time_one.drop('year_label', 1, inplace= True)
crimes_time_one.drop('DISTRICT', 1, inplace= True)
crimes_time_one.drop('Lat', 1, inplace= True)
crimes_time_one.drop('Long', 1, inplace= True)
crimes_time_one.drop('Lat_M', 1, inplace= True)
crimes_time_one.drop('Long_M', 1, inplace= True)

In [11]:
crimes_time_one = crimes_time_one.sort_values(['YEAR', 'MONTH', 'DAY_OF_MONTH', 'HOUR'], ascending=[True, True, True, True])
crimes_time_train_test = crimes_time_one.head(int(crimes_time_one.shape[0] * 0.8))
crimes_time_validation = crimes_time_one.tail(crimes_time_one.shape[0] - int(crimes_times.shape[0] * 0.8))
crimes_time_train_test.drop('YEAR', 1, inplace= True)
crimes_time_validation.drop('YEAR', 1, inplace= True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [12]:
crimes_time_train_test.head(5)

,Armas,Asesinato,Desaparecido,Desorden,Drogas,Falsificacion,Fraude,Hurto,Infancia,Investigar,...,Tuesday,Wednesday,SHOOTING,MONTH,DAY_OF_MONTH,HOUR,IS_NIGHT,TIME_X,TIME_Y,DISTRICT_labels
127685,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6,15,0,1,0.022469,0.648202,5
242709,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6,15,0,1,0.022469,0.648202,4
290054,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6,15,0,1,0.022469,0.648202,0
297104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6,15,0,1,0.022469,0.648202,5
301293,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,6,15,0,1,0.022469,0.648202,4


In [13]:
crimes_time_validation.head(5)

,Armas,Asesinato,Desaparecido,Desorden,Drogas,Falsificacion,Fraude,Hurto,Infancia,Investigar,...,Tuesday,Wednesday,SHOOTING,MONTH,DAY_OF_MONTH,HOUR,IS_NIGHT,TIME_X,TIME_Y,DISTRICT_labels
61632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2,3,15,0,0.918534,0.773549,5
61649,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2,3,15,0,0.918534,0.773549,1
61654,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2,3,15,0,0.918534,0.773549,6
61655,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,2,3,15,0,0.918534,0.773549,6
61658,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2,3,15,0,0.918534,0.773549,10


In [14]:
crimes_time_validation.shape

(61104, 31)

In [15]:
crimes_time_train_test.shape

(244416, 31)

In [16]:
xTrain, xTest, yTrain, yTest = train_test_split(crimes_time_train_test.loc[:,'Armas':'TIME_Y'], crimes_time_train_test.loc[:,'DISTRICT_labels'], test_size = 0.25, random_state = 0)
#xTrain, xTest, yTrain, yTest = train_test_split(crimes_time_train_test.loc[:,'OFFENSE_CODE_GROUP':'TIME_Y'], crimes_time_train_test.loc[:,'DISTRICT'], test_size = 0.25, random_state = 0)

In [17]:
#xTrain.drop([ 'TIME_X', 'TIME_Y'], 1, inplace= True)
#xTest.drop([ 'TIME_X', 'TIME_Y'], 1, inplace= True)
xTrain.drop([ 'MONTH', 'DAY_OF_MONTH', 'HOUR'], 1, inplace= True)
xTest.drop([ 'MONTH', 'DAY_OF_MONTH', 'HOUR'], 1, inplace= True)

In [18]:
xTrain.head(15)

,Armas,Asesinato,Desaparecido,Desorden,Drogas,Falsificacion,Fraude,Hurto,Infancia,Investigar,...,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,SHOOTING,IS_NIGHT,TIME_X,TIME_Y
212312,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0.011357,0.605961
142481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,0.554321,0.997040
100391,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.245870,0.069398
254792,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.999990,0.496772
197659,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.140833,0.152151
102119,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,0.197086,0.102202
251384,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.986711,0.614509
127959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.119590,0.824482
289106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0.141817,0.151137
191587,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0.302623,0.040606


In [19]:
def create_model (type_layer= 'LSTM', n_layers=1, n_embeding = 32,n_epochs = 20,batch_size=512 ):
    
    layer_r = tf.keras.layers.LSTM(n_embeding, return_sequences=True)
    layer_e = tf.keras.layers.LSTM(n_embeding)
    if type_layer == 'GRU':
        layer_r = tf.keras.layers.GRU(n_embeding, return_sequences=True)
        layer_e = tf.keras.layers.GRU(n_embeding)
        
    model_i = tf.keras.models.Sequential()
    model_i.add(tf.keras.layers.Embedding(30, n_embeding, input_length=27))
    if n_layers != 0:
            model_i.add(layer_r)
    
    model_i.add(layer_e)
    
    model_i.add(tf.keras.layers.Dense(12))
    optimizer = tf.keras.optimizers.Adam(lr=0.01)
    model_i.compile (optimizer=optimizer,loss='sparse_categorical_crossentropy', metrics=['accuracy'] )
    
    return model_i.fit(xTrain, yTrain, epochs = n_epochs,batch_size=batch_size, validation_data=(xTest, yTest))


In [51]:
create_model (type_layer= 'LSTM', n_layers=0, n_embeding = 128,n_epochs = 20,batch_size=256 )

Train on 183312 samples, validate on 61104 samples
Epoch 1/20
183312/183312 [==============================] - 7s 37us/sample - loss: 3.1297 - accuracy: 0.0483 - val_loss: 3.0661 - val_accuracy: 0.0208
Epoch 2/20
183312/183312 [==============================] - 5s 25us/sample - loss: 3.0467 - accuracy: 0.0805 - val_loss: 3.0414 - val_accuracy: 0.1113
Epoch 3/20
183312/183312 [==============================] - 5s 25us/sample - loss: 3.0240 - accuracy: 0.1408 - val_loss: 3.0771 - val_accuracy: 0.1396
Epoch 4/20
183312/183312 [==============================] - 5s 25us/sample - loss: 3.1270 - accuracy: 0.1319 - val_loss: 3.1259 - val_accuracy: 0.1586
Epoch 5/20
183312/183312 [==============================] - 5s 25us/sample - loss: 3.1787 - accuracy: 0.1394 - val_loss: 3.1435 - val_accuracy: 0.1586
Epoch 6/20
183312/183312 [==============================] - 5s 25us/sample - loss: 3.1014 - accuracy: 0.1466 - val_loss: 3.2123 - val_accuracy: 0.0644
Epoch 7/20
183312/183312 [=================

In [52]:
def grid_search(n_layers=[0,1,2,3,4], type_layers= ['GRU','LSTM'], n_embedings=[64,128,256], n_epochs = 30):
    i=0
    lista = {}
    for n_layer in n_layers:
        for type_l in type_layers:
            for n_embeding in n_embedings:
                print("************ NUEVA PRUEBA *************")
                print (type_l)
                print (n_layer)
                print (n_embeding)
                try:
                    fit = create_model (type_layer= type_l, n_layers=n_layer, n_embeding = n_embeding, n_epochs=n_epochs, batch_size=128)
                    print(fit)
                    print(i)
                    lista[str(i)] = "type_l=" + type_l + ",n_layer="+ str(n_layer) + ",n_embeding" + str(n_embeding) + ",lon=" + str(lon)
                    i=i+1
                except:
                    print("Imposible entrenar el modelos con los parametros siguientes")
    return lista

resultados = grid_search ()


************ NUEVA PRUEBA *************
GRU
0
64
Train on 183312 samples, validate on 61104 samples
Epoch 1/30
183312/183312 [==============================] - 7s 39us/sample - loss: 2.7769 - accuracy: 0.1309 - val_loss: 2.3560 - val_accuracy: 0.1474
Epoch 2/30
183312/183312 [==============================] - 6s 34us/sample - loss: 2.3569 - accuracy: 0.1533 - val_loss: 2.3566 - val_accuracy: 0.1396
Epoch 3/30
183312/183312 [==============================] - 6s 34us/sample - loss: 2.3571 - accuracy: 0.1465 - val_loss: 2.3542 - val_accuracy: 0.1586
Epoch 4/30
183312/183312 [==============================] - 6s 34us/sample - loss: 2.3614 - accuracy: 0.1533 - val_loss: 2.3556 - val_accuracy: 0.1587
Epoch 5/30
183312/183312 [==============================] - 6s 34us/sample - loss: 2.3557 - accuracy: 0.1515 - val_loss: 2.3511 - val_accuracy: 0.1587
Epoch 6/30
183312/183312 [==============================] - 6s 34us/sample - loss: 2.3574 - accuracy: 0.1541 - val_loss: 2.3593 - val_accuracy: 0

Epoch 23/30
183312/183312 [==============================] - 7s 37us/sample - loss: 4.1291 - accuracy: 0.1135 - val_loss: 4.1318 - val_accuracy: 0.1113
Epoch 24/30
183312/183312 [==============================] - 7s 37us/sample - loss: 4.1291 - accuracy: 0.1135 - val_loss: 4.1318 - val_accuracy: 0.1113
Epoch 25/30
183312/183312 [==============================] - 7s 36us/sample - loss: 4.1291 - accuracy: 0.1135 - val_loss: 4.1318 - val_accuracy: 0.1113
Epoch 26/30
183312/183312 [==============================] - 7s 36us/sample - loss: 4.1291 - accuracy: 0.1135 - val_loss: 4.1318 - val_accuracy: 0.1113
Epoch 27/30
183312/183312 [==============================] - 7s 36us/sample - loss: 4.1291 - accuracy: 0.1135 - val_loss: 4.1318 - val_accuracy: 0.1113
Epoch 28/30
183312/183312 [==============================] - 7s 37us/sample - loss: 4.1291 - accuracy: 0.1135 - val_loss: 4.1318 - val_accuracy: 0.1113
Epoch 29/30
183312/183312 [==============================] - 7s 37us/sample - loss: 4.12

Epoch 14/30
183312/183312 [==============================] - 6s 35us/sample - loss: 2.5905 - accuracy: 0.1496 - val_loss: 2.5877 - val_accuracy: 0.1396
Epoch 15/30
183312/183312 [==============================] - 6s 35us/sample - loss: 2.5924 - accuracy: 0.1505 - val_loss: 2.5880 - val_accuracy: 0.1478
Epoch 16/30
183312/183312 [==============================] - 6s 35us/sample - loss: 2.5915 - accuracy: 0.1507 - val_loss: 2.6052 - val_accuracy: 0.1475
Epoch 17/30
183312/183312 [==============================] - 6s 35us/sample - loss: 2.5439 - accuracy: 0.0776 - val_loss: 2.4849 - val_accuracy: 0.0208
Epoch 18/30
183312/183312 [==============================] - 6s 35us/sample - loss: 2.4849 - accuracy: 0.0208 - val_loss: 2.4849 - val_accuracy: 0.0208
Epoch 19/30
183312/183312 [==============================] - 6s 35us/sample - loss: 2.4849 - accuracy: 0.0208 - val_loss: 2.4849 - val_accuracy: 0.0208
Epoch 20/30
183312/183312 [==============================] - 6s 35us/sample - loss: 2.48

Epoch 5/30
183312/183312 [==============================] - 11s 58us/sample - loss: 14.0052 - accuracy: 0.1306 - val_loss: 14.0555 - val_accuracy: 0.1274
Epoch 6/30
183312/183312 [==============================] - 11s 58us/sample - loss: 14.0052 - accuracy: 0.1306 - val_loss: 14.0555 - val_accuracy: 0.1274
Epoch 7/30
183312/183312 [==============================] - 11s 58us/sample - loss: 14.0052 - accuracy: 0.1306 - val_loss: 14.0555 - val_accuracy: 0.1274
Epoch 8/30
183312/183312 [==============================] - 11s 58us/sample - loss: 14.0052 - accuracy: 0.1306 - val_loss: 14.0555 - val_accuracy: 0.1274
Epoch 9/30
183312/183312 [==============================] - 11s 59us/sample - loss: 14.0052 - accuracy: 0.1306 - val_loss: 14.0555 - val_accuracy: 0.1274
Epoch 10/30
183312/183312 [==============================] - 11s 58us/sample - loss: 14.0052 - accuracy: 0.1306 - val_loss: 14.0555 - val_accuracy: 0.1274
Epoch 11/30
183312/183312 [==============================] - 11s 58us/sampl

Epoch 27/30
183312/183312 [==============================] - 8s 44us/sample - loss: 2.4849 - accuracy: 0.0546 - val_loss: 2.4849 - val_accuracy: 0.0571
Epoch 28/30
183312/183312 [==============================] - 8s 44us/sample - loss: 2.4849 - accuracy: 0.0546 - val_loss: 2.4849 - val_accuracy: 0.0571
Epoch 29/30
183312/183312 [==============================] - 8s 44us/sample - loss: 2.4849 - accuracy: 0.0546 - val_loss: 2.4849 - val_accuracy: 0.0571
Epoch 30/30
183312/183312 [==============================] - 8s 44us/sample - loss: 2.4849 - accuracy: 0.0546 - val_loss: 2.4849 - val_accuracy: 0.0571
0
Imposible entrenar el modelos con los parametros siguientes
************ NUEVA PRUEBA *************
GRU
1
128
Train on 183312 samples, validate on 61104 samples
Epoch 1/30
183312/183312 [==============================] - 11s 58us/sample - loss: 4.3134 - accuracy: 0.1529 - val_loss: 4.2727 - val_accuracy: 0.1586
Epoch 2/30
183312/183312 [==============================] - 9s 49us/sample - 

...

183312/183312 [==============================] - 15s 82us/sample - loss: 11.0676 - accuracy: 0.1553 - val_loss: 11.0059 - val_accuracy: 0.1586
Epoch 17/30
183312/183312 [==============================] - 16s 85us/sample - loss: 11.0676 - accuracy: 0.1553 - val_loss: 11.0059 - val_accuracy: 0.1586
Epoch 18/30
183312/183312 [==============================] - 16s 87us/sample - loss: 11.0676 - accuracy: 0.1553 - val_loss: 11.0059 - val_accuracy: 0.1586
Epoch 19/30
183312/183312 [==============================] - 16s 88us/sample - loss: 11.0676 - accuracy: 0.1553 - val_loss: 11.0059 - val_accuracy: 0.1586
Epoch 20/30
183312/183312 [==============================] - 16s 88us/sample - loss: 11.0676 - accuracy: 0.1553 - val_loss: 11.0059 - val_accuracy: 0.1586
Epoch 21/30
183312/183312 [==============================] - 16s 88us/sample - loss: 11.0676 - accuracy: 0.1553 - val_loss: 11.0059 - val_accuracy: 0.1586
Epoch 22/30
183312/183312 [==============================] - 16s 88us/sample - los

183312/183312 [==============================] - 18s 100us/sample - loss: 5.4689 - accuracy: 0.0628 - val_loss: 5.4855 - val_accuracy: 0.0605
Epoch 29/30
183312/183312 [==============================] - 17s 94us/sample - loss: 5.4689 - accuracy: 0.0628 - val_loss: 5.4855 - val_accuracy: 0.0605
Epoch 30/30
183312/183312 [==============================] - 18s 100us/sample - loss: 5.4689 - accuracy: 0.0628 - val_loss: 5.4855 - val_accuracy: 0.0605
0
Imposible entrenar el modelos con los parametros siguientes
************ NUEVA PRUEBA *************
GRU
4
64
Train on 183312 samples, validate on 61104 samples
Epoch 1/30
183312/183312 [==============================] - 11s 59us/sample - loss: 6.0729 - accuracy: 0.1115 - val_loss: 5.8562 - val_accuracy: 0.1113
Epoch 2/30
183312/183312 [==============================] - 9s 47us/sample - loss: 5.8821 - accuracy: 0.1135 - val_loss: 5.8562 - val_accuracy: 0.1113
Epoch 3/30
183312/183312 [==============================] - 8s 46us/sample - loss: 5.8

Epoch 19/30
183312/183312 [==============================] - 9s 49us/sample - loss: 4.9009 - accuracy: 0.1074 - val_loss: 4.9545 - val_accuracy: 0.1077
Epoch 20/30
183312/183312 [==============================] - 9s 50us/sample - loss: 4.9009 - accuracy: 0.1074 - val_loss: 4.9545 - val_accuracy: 0.1077
Epoch 21/30
183312/183312 [==============================] - 9s 50us/sample - loss: 4.9009 - accuracy: 0.1074 - val_loss: 4.9545 - val_accuracy: 0.1077
Epoch 22/30
183312/183312 [==============================] - 9s 49us/sample - loss: 4.9009 - accuracy: 0.1074 - val_loss: 4.9545 - val_accuracy: 0.1077
Epoch 23/30
183312/183312 [==============================] - 9s 49us/sample - loss: 4.9009 - accuracy: 0.1074 - val_loss: 4.9545 - val_accuracy: 0.1077
Epoch 24/30
183312/183312 [==============================] - 9s 50us/sample - loss: 4.9009 - accuracy: 0.1074 - val_loss: 4.9545 - val_accuracy: 0.1077
Epoch 25/30
183312/183312 [==============================] - 9s 50us/sample - loss: 4.90

Epoch 10/30
183312/183312 [==============================] - 9s 47us/sample - loss: 2.5926 - accuracy: 0.1487 - val_loss: 2.5855 - val_accuracy: 0.1586
Epoch 11/30
183312/183312 [==============================] - 9s 48us/sample - loss: 2.5937 - accuracy: 0.1500 - val_loss: 2.5911 - val_accuracy: 0.1586
Epoch 12/30
183312/183312 [==============================] - 9s 47us/sample - loss: 2.5925 - accuracy: 0.1501 - val_loss: 2.5859 - val_accuracy: 0.1586
Epoch 13/30
183312/183312 [==============================] - 9s 47us/sample - loss: 2.5920 - accuracy: 0.1491 - val_loss: 2.5857 - val_accuracy: 0.1396
Epoch 14/30
183312/183312 [==============================] - 9s 47us/sample - loss: 2.5922 - accuracy: 0.1493 - val_loss: 2.5862 - val_accuracy: 0.1586
Epoch 15/30
183312/183312 [==============================] - 9s 47us/sample - loss: 2.5923 - accuracy: 0.1498 - val_loss: 2.5905 - val_accuracy: 0.1586
Epoch 16/30
183312/183312 [==============================] - 9s 47us/sample - loss: 2.59

Train on 183312 samples, validate on 61104 samples
Epoch 1/30
183312/183312 [==============================] - 18s 99us/sample - loss: 11.1927 - accuracy: 0.1383 - val_loss: 11.2185 - val_accuracy: 0.1396
Epoch 2/30
183312/183312 [==============================] - 16s 89us/sample - loss: 11.1875 - accuracy: 0.1387 - val_loss: 11.2185 - val_accuracy: 0.1396
Epoch 3/30
183312/183312 [==============================] - 16s 89us/sample - loss: 11.1875 - accuracy: 0.1387 - val_loss: 11.2185 - val_accuracy: 0.1396
Epoch 4/30
183312/183312 [==============================] - 16s 89us/sample - loss: 11.1875 - accuracy: 0.1387 - val_loss: 11.2185 - val_accuracy: 0.1396
Epoch 5/30
183312/183312 [==============================] - 16s 90us/sample - loss: 11.1875 - accuracy: 0.1387 - val_loss: 11.2185 - val_accuracy: 0.1396
Epoch 6/30
183312/183312 [==============================] - 17s 90us/sample - loss: 11.1875 - accuracy: 0.1387 - val_loss: 11.2185 - val_accuracy: 0.1396
Epoch 7/30
183312/183312 

In [54]:
n_embeding=20

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(500, activation='relu', input_dim=27))
model.add(tf.keras.layers.Dense(250, activation='relu'))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(12, activation='softmax'))
model.compile (optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'] )
model_fitted_3 = model.fit(xTrain, yTrain, epochs = 50,batch_size=128, validation_data=(xTest, yTest))

Train on 183312 samples, validate on 61104 samples
Epoch 1/50
183312/183312 [==============================] - 3s 16us/sample - loss: 2.3323 - accuracy: 0.1710 - val_loss: 2.3261 - val_accuracy: 0.1753
Epoch 2/50
183312/183312 [==============================] - 3s 15us/sample - loss: 2.3256 - accuracy: 0.1754 - val_loss: 2.3261 - val_accuracy: 0.1775
Epoch 3/50
183312/183312 [==============================] - 3s 14us/sample - loss: 2.3238 - accuracy: 0.1754 - val_loss: 2.3238 - val_accuracy: 0.1788
Epoch 4/50
183312/183312 [==============================] - 3s 14us/sample - loss: 2.3227 - accuracy: 0.1762 - val_loss: 2.3240 - val_accuracy: 0.1785
Epoch 5/50
183312/183312 [==============================] - 3s 14us/sample - loss: 2.3214 - accuracy: 0.1774 - val_loss: 2.3237 - val_accuracy: 0.1795
Epoch 6/50
183312/183312 [==============================] - 3s 14us/sample - loss: 2.3202 - accuracy: 0.1776 - val_loss: 2.3234 - val_accuracy: 0.1793
Epoch 7/50
183312/183312 [=================

In [ ]:
n_embeding=20

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(500, activation='relu', input_dim=27))
model.add(tf.keras.layers.Dense(250, activation='relu'))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(12, activation='softmax'))
model.compile (optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'] )
model_fitted_3 = model.fit(xTrain, yTrain, epochs = 50,batch_size=128, validation_data=(xTest, yTest))

Train on 183312 samples, validate on 61104 samples
Epoch 1/50
183312/183312 [==============================] - 3s 16us/sample - loss: 2.3326 - accuracy: 0.1705 - val_loss: 2.3282 - val_accuracy: 0.1750
Epoch 2/50
183312/183312 [==============================] - 3s 14us/sample - loss: 2.3260 - accuracy: 0.1745 - val_loss: 2.3248 - val_accuracy: 0.1770
Epoch 3/50
183312/183312 [==============================] - 3s 14us/sample - loss: 2.3241 - accuracy: 0.1754 - val_loss: 2.3244 - val_accuracy: 0.1750
Epoch 4/50
183312/183312 [==============================] - 3s 15us/sample - loss: 2.3229 - accuracy: 0.1761 - val_loss: 2.3236 - val_accuracy: 0.1773
Epoch 5/50
183312/183312 [==============================] - 3s 14us/sample - loss: 2.3216 - accuracy: 0.1759 - val_loss: 2.3230 - val_accuracy: 0.1782
Epoch 6/50
183312/183312 [==============================] - 3s 14us/sample - loss: 2.3203 - accuracy: 0.1770 - val_loss: 2.3242 - val_accuracy: 0.1768
Epoch 7/50
183312/183312 [=================

In [ ]:
#Padding the sequence with the values in last row to max length
to_pad = 129
new_seq = []
for one_seq in sequences:
    len_one_seq = len(one_seq)
    last_val = one_seq[-1]
    n = to_pad - len_one_seq
   
    to_concat = np.repeat(one_seq[-1], n).reshape(4, n).transpose()
    new_one_seq = np.concatenate([one_seq, to_concat])
    new_seq.append(new_one_seq)
final_seq = np.stack(new_seq)

#truncate the sequence to length 60
from keras.preprocessing import sequence
seq_len = 60
final_seq=sequence.pad_sequences(final_seq, maxlen=seq_len, padding='post', dtype='float', truncating='post')

In [ ]:
n_embeding=20

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(100, activation='relu', input_dim=79))
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(12, activation='softmax'))
model.compile (optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'] )
model_fitted_3 = model.fit(xTrain, yTrain, epochs = 20,batch_size=128, validation_data=(xTest, yTest))

In [ ]:
model_fitted_3 = model.fit(xTrain, yTrain, epochs = 50,batch_size=128, validation_data=(xTest, yTest))

In [ ]:
create_model (type_layer= 'GRU', n_layers=2, n_embeding = 100,n_epochs = 20,batch_size=256 )

In [ ]:
create_model (type_layer= 'GRU', n_layers=0, n_embeding = 100,n_epochs = 50,batch_size=128 )

In [ ]:
create_model (type_layer= 'GRU', n_layers=0, n_embeding = 300,n_epochs = 50,batch_size=128 )

In [ ]:
create_model (type_layer= 'LSTM', n_layers=0, n_embeding = 300,n_epochs = 50,batch_size=128 )

In [ ]:
create_model (type_layer= 'LSTM', n_layers=5, n_embeding = 50,n_epochs = 50,batch_size=128 )

In [ ]:
model_fitted_1 = create_model (type_layer= 'LSTM', n_layers=3, n_embeding = 70,n_epochs = 30,batch_size=128 )
model_fitted_2 = create_model (type_layer= 'GRU', n_layers=3, n_embeding = 70,n_epochs = 30,batch_size=128 )

In [ ]:
n_embeding=20

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(500, activation='relu', input_dim=85))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(12, activation='softmax'))
model.compile (optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'] )
model_fitted_3 = model.fit(xTrain, yTrain, epochs = 20,batch_size=128, validation_data=(xTest, yTest))

In [ ]:
model__2 = Sequential()
model__2.add(LSTM(20, input_shape=(82, 1), return_sequences=True))
model__2.add(TimeDistributed(Dense(12, activation='sigmoid')))
model__2.compile (optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'] )
model__2 = model.fit(xTrain, yTrain, epochs = 20,batch_size=128, validation_data=(xTest, yTest))

In [ ]:
def get_sequence(n_timesteps):
    # create a sequence of random numbers in [0,1]
    X = array([random() for _ in range(n_timesteps)])
    # calculate cut-off value to change class values
    limit = n_timesteps/4.0
    # determine the class outcome for each item in cumulative sequence
    y = array([0 if x < limit else 1 for x in cumsum(X)])
    # reshape input and output data to be suitable for LSTMs
    X = X.reshape(1, n_timesteps, 1)
    y = y.reshape(1, n_timesteps, 1)
    return X, y
 
# define problem properties
n_timesteps = 10
# define LSTM
model = Sequential()
model.add(LSTM(20, input_shape=(n_timesteps, 1), return_sequences=True))
model.add(TimeDistributed(Dense(12, activation='sigmoid')))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# train LSTM
for epoch in range(1000):
    # generate new random sequence
    X,y = get_sequence(n_timesteps)
    # fit model for one epoch on this sequence
    model.fit(X, y, epochs=1, batch_size=1, verbose=2)
# evaluate LSTM
X,y = get_sequence(n_timesteps)
yhat = model.predict_classes(X, verbose=0)
for i in range(n_timesteps):
    print('Expected:', y[0, i], 'Predicted', yhat[0, i])